 # 机器学习练习3-多类分类

使用python语言完成作业

代码修改并注释：Changersh，Changersh@outlook.com

# 1、多类分类

In [123]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

## 1.1 读入数据

数据以 .mat 类型存储，python中使用 `lodamat` 读取，是scipy.io中的工具
.mat 文件会以矩阵的格式保存

In [124]:
path = 'ex3data1.mat'
data = loadmat(path)
data
# 注意，此数据原用于 matlab，为了避免 0 索引，就将y中是0的结果，使用 10 代替。

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

## 1.2 可视化数据
没有对应的matlab函数，放弃了

## 1.3 向量化逻辑回归

### 1.3.1 向量化Cost函数
首先是 z 形函数

In [125]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

逻辑回归的代价函数：
$$
J(\theta)= - \frac{1}{m}\sum_{i=1}^m[y^{(i)}*log(h_\theta(x^{i})) + (1-y^{i})*log(1-h_\theta(x^{i}))] + \frac{\lambda}{2m}\sum_{j=1}^n \theta_j^2
$$

In [126]:
# 使用正则化之后的函数，没有 for 循环
def cost(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / (2 * len(X))) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / len(X) + reg

### 1.3.2 向量化梯度函数gradient

lam就是上面式子中的λ，是正则化参数，不能太大，否则会导致拟合效果很差。
我们也要写出更新参数 θ 的函数，并且不用给出 学习速率α，因为我们后面使用第一题使用过的高级的方法，会自动给出学习速率
$$
Repeat \ until \ convergence \{ \\
\theta_0 := \theta_0 - \alpha \frac{1}{m}\sum_{i=1}^m(h_\theta(x^{(i)}) -y^{(i)})x_0^{(i)}
\\
\theta_j := \theta_j - \alpha [\frac{1}{m}\sum_{i=1}^m(h_\theta(x^{(i)}) -y^{(i)})x_j^{(i)} + \frac{\lambda}{m}\theta_j]
\\
\}
$$

后面一项可以化简
$$
\theta_j := \theta_j(1-\alpha \frac{\lambda}{m}) - \alpha \frac{1}{m}\sum_{i=1}^m(h_\theta(x^{(i)}) -y^{(i)})x_j^{(i)}
$$

使用 for 循环的梯度函数

In [127]:
def gradient_with_loop(theta, X, y, lam):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = X.shape[1]
    grad = np.zeros(parameters) # 存储梯度

    error = sigmoid(X * theta.T) - y # 这一项跟theta没关系，预处理出来
    for i in range(parameters):
        tmp = np.sum(np.multiply(error, X[:, i])) / X.shape[0]
        if i == 0:
            grad[i] = tmp
        else:
            grad[i] = tmp + (lam * theta[:, i]) / X.shape[0]

    return grad

向量化的梯度函数

In [128]:
def gradient(theta, X, y, lam):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = int(theta.ravel().shape[1])

    error = sigmoid(X * theta.T) - y # 这一项跟theta没关系，预处理出来

    grad = ((X.T * error) / X.shape[0]).T + ((lam / X.shape[0]) * theta)
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / X.shape[0]

    return np.array(grad).ravel()

## 1.4 One-vs-All分类

我们有十个可能的类，逻辑回归只能在两个类之间分类。多类分类的策略就是分为 是i 和 不是i 两种，最后逐个套用分类器，比较概率。

这样的话，循环进行分类，最后比较即可

In [129]:
from scipy.optimize import minimize
# fmin_tnc 拟牛顿法用于最小优化问题
def one_vs_all(X, y, num_labels, lam):
    rows = X.shape[0]
    params = X.shape[1]

    # k * (n+1) k个分类器中，每个分类器的参数 theta
    all_theta = np.zeros((num_labels, params + 1))

    # 在 0 位置插入长度是 rows 的 1 ，以 axis=1（列）的形式
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # 切记循环从 1 开始，0被映射为 10 了
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))

        # 最小化函数求 theta
        # x0是要优化的变量的初始值
        # TNC是优化方法，截断牛顿法
        # jac是梯度函数
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, lam), method='TNC', jac=gradient)

        all_theta[i-1, :] = fmin.x

    return all_theta

初始化各种向量，注意维度

In [130]:
rows = data['X'].shape[0]
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

theta = np.zeros(params + 1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [131]:
np.unique(data['y'])#看下有几类标签

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

确保训练函数正确运行，得到合理的输出

In [132]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[-2.38090200e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30362219e-03, -6.06149084e-10,  0.00000000e+00],
       [-3.18420805e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.46211711e-03, -5.08723221e-04,  0.00000000e+00],
       [-4.79563739e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.86475853e-05, -2.47347362e-07,  0.00000000e+00],
       ...,
       [-7.98603898e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.94272873e-05,  7.21573274e-06,  0.00000000e+00],
       [-4.57361264e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33685653e-03,  1.00046175e-04,  0.00000000e+00],
       [-5.40406502e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.16595999e-04,  7.89322344e-06,  0.00000000e+00]])

验证正确性之后，使用训练完毕的分类器预测每个图象的标签。
我们算每个类的概率，输出标签为概率最高的类

预测函数$h_\theta = g(\theta^TX)$
预测出的是 正确的概率

In [141]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]

    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)

    h = sigmoid(X * all_theta.T) # 5000*10 是每个例子中，是数字 i 的概率

    # 获得 h 中最大元素的索引
    h_argmax = np.argmax(h, axis=1)

    h_argmax = h_argmax + 1

    return h_argmax

In [142]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

(5000, 10)
